## Empty server

In [1]:
from nanover.omni import OmniRunner

runner = OmniRunner.with_basic_server()
runner.print_basic_info()

Serving "ragzo: NanoVer iMD Server" (ws://localhost:38801), discoverable on all interfaces on port 54545
Available simulations:



## Client connected to empty server

In [2]:
from nanover.websocket import NanoverImdClient

client = NanoverImdClient.from_runner(runner)

## Simulation

In [3]:
from nanover.openmm import OpenMMSimulation

input_xml = "openmm_files/nanotube.xml"
nanotube_simulation = OpenMMSimulation.from_xml_path(input_xml)

## Publish simulation frames via client

In [4]:
# NBVAL_SKIP
from contextlib import suppress
from nanover.app import NanoverImdApplication
from nanover.utilities.timing import yield_interval

fake_app_server = NanoverImdApplication.null_server()
fake_runner = OmniRunner(fake_app_server)
fake_runner.add_simulation(nanotube_simulation)

fake_runner.load(0)
fake_runner.print_basic_info()

def send_frame(frame_index, frame):
    frame.frame_index = frame_index
    message = {
        "frame": frame.pack_to_dict(),
    }
    client.send_message(message)

nanotube_simulation.load()
nanotube_simulation.reset(fake_app_server)
frame = nanotube_simulation.make_topology_frame()
send_frame(0, frame)

with suppress(KeyboardInterrupt):
    for dt in yield_interval(1/30):
        nanotube_simulation.advance_by_one_step()
        frame = nanotube_simulation.make_regular_frame()
        send_frame(1, frame)

Serving "ragzo: NanoVer iMD Server" (), without discovery
Available simulations:
0: "nanotube"
